In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from PIL import Image
from sklearn.metrics import confusion_matrix
from random import choice,sample
from ipynb.fs.full.create_dataframe import read_data



/Users/erikafox/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def center_crop(im, crop_pixels = 10):
    return im[crop_pixels:im.shape[0] - crop_pixels, crop_pixels:im.shape[1] - crop_pixels]

In [ ]:
def basic_preprocess(dataframe):
    # Basic preprocessing 
    dataframe=dataframe[dataframe.astype(str)['image'] != '[]']
    # prepare X
    img_list = list(dataframe['image'])
    data_img = []
    for each in img_list:
                each_img = cv2.cvtColor(each, cv2.COLOR_BGR2RGB)
                #Resize the images:
                each_img_resized = cv2.resize(each_img, (128,128))

                each_img_cropped=center_crop(each_img_resized)

        
                #Save arrays to a list:
                data_img.append(each_img_cropped)
    # Converting list to numpy array
    X = np.array(data_img)

    # prepare y
    y = OneHotEncoder(dtype='int8', sparse=False).fit_transform(dataframe['target'].values.reshape(-1,1))

    return X,y

def fit_model(X_train,y_train,X_val,y_val):
    #  code adapted from:
    #  https://pub.towardsai.net/multiclass-image-classification-hands-on-with-keras-and-tensoflow-e1cf434f3467


    train_gen = ImageDataGenerator(horizontal_flip=True,
                               rotation_range = 45,
                               zoom_range=0.2,
                               height_shift_range = 0.5,
                               width_shift_range = 0.5)
    validation_gen = ImageDataGenerator(horizontal_flip=True,
                                    rotation_range = 45,
                                    zoom_range=0.2,
                                    height_shift_range = 0.5,
                                    width_shift_range = 0.5)
    train_gen.fit(X_train)
    validation_gen.fit(X_val)
    batch_size = 64
    epochs = 10
    base = Xception(include_top=False, 
                weights='imagenet', 
                input_shape=X_train[1].shape)
    x = base.output
    x = GlobalAveragePooling2D()(x)
    head = Dense(y_train.shape[1], activation='softmax')(x)
    model = Model(inputs=base.input, outputs=head)
    model.compile(optimizer=Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'], run_eagerly=True)
    
    fit = model.fit_generator(
               train_gen.flow(X_train, y_train,
               batch_size=batch_size),
               epochs = epochs,
               validation_data = validation_gen.flow(X_val, y_val)
)

    return model,fit


In [ ]:
def bright_contrast (image, alpha=1, beta=0):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

In [ ]:
reptiles=read_data(["Lizard","Toad"])

In [ ]:
reptiles["image"]=reptiles["image"].apply(lambda x: bright_contrast(x))

In [ ]:
reptiles['image'][1].shape

(200, 300, 4)

In [ ]:
X,y= basic_preprocess(reptiles) ## replace this to change preprocessing

X_data, X_test, y_data, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_val shape  : ', X_val.shape)
print('y_val shape  : ', y_val.shape)
print('X_test shape : ', X_test.shape)
print('y_test shape : ', y_test.shape)

X_train shape:  (2460, 108, 108, 3)
y_train shape:  (2460, 8)
X_val shape  :  (616, 108, 108, 3)
y_val shape  :  (616, 8)
X_test shape :  (543, 108, 108, 3)
y_test shape :  (543, 8)


In [ ]:
X[1].shape


(108, 108, 3)

In [ ]:
model,fit=fit_model(X_train,y_train,X_val,y_val)

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47604/661476147.py:55: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit = model.fit_generator(


Epoch 1/10
39/39 [==============================] - 495s 13s/step - loss: 1.9340 - accuracy: 0.2687 - val_loss: 1.9694 - val_accuracy: 0.2955
Epoch 2/10
39/39 [==============================] - 467s 12s/step - loss: 1.4916 - accuracy: 0.5065 - val_loss: 1.6454 - val_accuracy: 0.4659
Epoch 3/10
39/39 [==============================] - 664s 17s/step - loss: 1.1563 - accuracy: 0.6098 - val_loss: 1.2800 - val_accuracy: 0.5877
Epoch 4/10
39/39 [==============================] - 580s 15s/step - loss: 0.9732 - accuracy: 0.6764 - val_loss: 1.0492 - val_accuracy: 0.6640
Epoch 5/10
39/39 [==============================] - 547s 14s/step - loss: 0.8245 - accuracy: 0.7175 - val_loss: 0.9415 - val_accuracy: 0.7029
Epoch 6/10
39/39 [==============================] - 521s 13s/step - loss: 0.7823 - accuracy: 0.7333 - val_loss: 0.9762 - val_accuracy: 0.6640
Epoch 7/10
39/39 [==============================] - 522s 13s/step - loss: 0.6645 - accuracy: 0.7809 - val_loss: 0.8722 - val_accuracy: 0.7110
Epoch 